## Quantifiers and scope tutorial

### Kyle Rawlins  11/16/18, updated 10/27/2021

This notebook walks through several basic techniques for handling quantifier scope, together with how to implement them in the Lambda Notebook.

### Quantifiers in object position


In [ ]:
%%lamb
||every|| = L f_<e,t> : L g_<e,t> : Forall x_e : f(x) >> g(x)
||doctor|| = L x_e : Doctor(x)
||someone|| = L f_<e,t> : Exists x_e : Human(x) & f(x)
||saw|| = L x_e : L y_e : Saw(y,x)
||alfonso|| = Alfonso_e

In [ ]:
((every * doctor) * (saw * alfonso)).tree()

In [ ]:
(saw * (every * doctor))

### Quantifiers in object position via QR

The "standard" approach is to move an object position quantified DP so it scopes over its immediate TP.  The lambda notebook doesn't automate QR yet (feel free to submit a pull request...).  However, it is easy to construct the trees "by hand".  This notebook does so by bottom-up composition. We will need traces and binders.  A version of Predicate Abstraction (PA) is already present as a composition operation in the default system.

In [ ]:
trace = lang.Trace(2)
trace

In [ ]:
binder = lang.Binder(2)
binder

In [ ]:
lang.get_system()

In [ ]:
# use svgling to at least produce some nice diagrams -- some day this can be auto-generated!
import svgling, svgling.figure
qrtree2 = ("TP", ("DP", ("D", "every"), ("NP", ("N", "doctor"))),
                      ("TP", "3", ("TP", ("DP", ("D", "Alfonso")),
                                   ("VP", ("V", "saw"), ("DP", "t(3)")))))
out2 = svgling.draw_tree(qrtree2)
out2.movement_arrow((1,1,1,1), (0,))
out2.box_constituent((0,))
out2 = svgling.figure.Caption(out2, "Object position QR, referential subject")
out2

In [ ]:
objqr = (every * doctor) * (binder * (alfonso * (saw * trace)))
display(objqr.source_tree())
objqr

In [ ]:
((every * doctor) * (binder * (alfonso * (saw * trace)))).tree()

In [ ]:
import svgling, svgling.figure
qrtree2 = ("TP", ("DP", ("D", "every"), ("NP", ("N", "doctor"))),
                      ("TP", "3", ("TP", ("DP", ("D", "someone")),
                                   ("VP", ("V", "saw"), ("DP", "t(3)")))))
out2 = svgling.draw_tree(qrtree2)
out2.movement_arrow((1,1,1,1), (0,))
out2.box_constituent((0,))
out2 = svgling.figure.Caption(out2, "Object position QR, quantificational subject")
out2

In [ ]:
inverse = ((every * doctor) * (binder * (someone * (saw * trace))))
display(inverse.source_tree())
inverse

To get surface scope, you need to also move the subject (after moving the object).  This results in a second trace / binding operator.

In [ ]:
import svgling, svgling.figure
qrtree3 = ("TP", ("DP", ("D", "every"), ("NP", ("N", "cat"))),
           ("TP", "5", ("TP", ("DP", ("D", "someone")),
                     ("TP", "2", ("TP", ("DP", "t(5)"), ("VP", ("V", "likes"), ("DP", "t(2)")))))))
out3 = svgling.draw_tree(qrtree3)
out3.movement_arrow((1,1,1,1,0), (0,))
out3.box_constituent((0,))
out3.movement_arrow((1,1,1,1,1,1), (1,1,0))
out3.box_constituent((1,1,0))
out3 = svgling.figure.Caption(out3, "QR object then subject")
out3

In [ ]:
surface = (someone * (lang.Binder(5) * ((every * doctor) * (binder * (lang.Trace(5) * (saw * trace))))))
display(surface.source_tree())
surface

Here's another example illustrating the other QR order that is possible, where first the subject QRs and then the object. This is equivalent to interpreting the subject in situ.

In [ ]:
%%lamb
||every|| = L f_<e,t> : L g_<e,t> : Forall x_e : f(x) >> g(x)
||cat|| = L x_e : Cat(x)
||dog|| = L x_e : Dog(x)
||some|| = L f_<e,t> : L g_<e,t> : Exists x_e : f(x) & g(x)
||likes|| = L x_e : L y_e : Likes(y,x)

In [ ]:
import svgling, svgling.figure
qrtree3 = ("TP", ("DP", ("D", "every"), ("NP", ("N", "cat"))),
           ("TP", "1", ("TP", ("DP", ("D", "some"), ("NP", ("N", "dog"))),
                     ("TP", "3", ("TP", ("DP", "t(3)"), ("VP", ("V", "likes"), ("DP", "t(1)")))))))
out3 = svgling.draw_tree(qrtree3)
out3.movement_arrow((1,1,1,1,1,1), (0,))
out3.box_constituent((0,))
#out3.movement_arrow((1,1,1,1,1,1), (1,1,0))
out3.movement_arrow((1,1,1,1,0), (1,1,0))
out3.box_constituent((1,1,0))
out3 = svgling.figure.Caption(out3, "QR subject then object")
out3

In [ ]:
(every * cat) * (lang.Binder(1) * ((some * dog) * (lang.Binder(3) *  (lang.Trace(1) * (likes * lang.Trace(3))))))

In [ ]:
((every * cat) * (lang.Binder(1) * ((some * dog) * (lang.Binder(3) *  (lang.Trace(3) * (likes * lang.Trace(1))))))).tree()

Summary figure showing a complete QR process. This figure is adapted directly from the svgling gallery (https://nbviewer.org/github/rawlins/svgling/blob/master/svgling%20Gallery.ipynb).

In [ ]:
import svgling, svgling.figure
qrtree0 = ("TP", ("DP", ("D", "every"), ("NP", ("N", "cat"))),
                                   ("VP", ("V", "likes"), ("DP", ("D", "some"), ("NP", ("N", "dog")))))
out0 = svgling.figure.Caption(svgling.draw_tree(qrtree0), "LF input (= Surface Structure)")

qrtree1 = ("TP", ("DP(1)", ("D", "every"), ("NP", ("N", "cat"))),
                                   ("VP", ("V", "likes"), ("DP(3)", ("D", "some"), ("NP", ("N", "dog")))))
out1 = svgling.draw_tree(qrtree1)
out1.box_constituent((0,))
out1.box_constituent((1,1))
out1 = svgling.figure.Caption(out1, "Step 1: free indexing of DPs (1 of 2 indexings)")

qrtree2 = ("TP", ("DP", ("D", "some"), ("NP", ("N", "dog"))),
                      ("TP", "3", ("TP", ("DP(1)", ("D", "every"), ("NP", ("N", "cat"))),
                                   ("VP", ("V", "likes"), ("DP", "t(3)")))))
out2 = svgling.draw_tree(qrtree2)
out2.movement_arrow((1,1,1,1), (0,))
out2.box_constituent((0,))
out2.box_constituent((1,1,0))
out2 = svgling.figure.Caption(out2, "Step 2: QR an indexed DP (choosing the object)")

qrtree3 = ("TP", ("DP", ("D", "every"), ("NP", ("N", "cat"))),
           ("TP", "1", ("TP", ("DP", ("D", "some"), ("NP", ("N", "dog"))),
                     ("TP", "3", ("TP", ("DP", "t(1)"), ("VP", ("V", "likes"), ("DP", "t(3)")))))))
out3 = svgling.draw_tree(qrtree3)
out3.movement_arrow((1,1,1,1,0), (0,))
out3.box_constituent((0,))
out3.movement_arrow((1,1,1,1,1,1), (1,1,0))
out3.box_constituent((1,1,0))
out3 = svgling.figure.Caption(out3, "Step 3: QR an indexed DP (choosing the subject).")

svgling.figure.Caption(svgling.figure.RowByRow(svgling.figure.SideBySide(out0, out1), svgling.figure.SideBySide(out2,out3)), "Trees illustrating a QR (Quantifier Raising) derivation in the Heim & Kratzer 1998 style")

### Quantifiers in object position via type shifting

An alternative approach to the object-position puzzle is to type-shift the DP to a type where it can take a transitive predicate and ignore the external argument position.

This would need to be generalized for n-ary predicates, e.g. the indirect object of "introduce".

In [ ]:
%%lamb
||every|| = L f_<e,t> : L g_<e,t> : Forall x_e : f(x) >> g(x)
||doctor|| = L x_e : Doctor(x)
||someone|| = L f_<e,t> : Exists x_e : Human(x) & f(x)
||saw|| = L x_e : L y_e : Saw(y,x)
||alfonso|| = Alfonso_e

The following combinator shifts a GQ type into something that can handle a transitive verb.

In [ ]:
gq_lift_combinator = %te L f_<<e,t>,t> : L g_<e,<e,t>> : L x_e : f(L y_e : g(y)(x))
gq_lift_combinator

In [ ]:
gq_lift_combinator.type

In [ ]:
gq_lift_combinator(someone.content).reduce_all()

In [ ]:
system = lang.td_system.copy()
system.add_rule(lang.unary_factory(gq_lift_combinator, "gq-lift-trans", typeshift=True))
system.typeshift = True
lang.set_system(system)
system

In [ ]:
display((alfonso * (saw * someone)).source_tree())
(alfonso * (saw * someone))

In [ ]:
(alfonso * (saw * someone)).tree()

In [ ]:
display((someone * (saw * (every * doctor))).source_tree())
(someone * (saw * (every * doctor)))

In [ ]:
((every * doctor) * (saw * someone))

### Quantifier scope via type shifting

This so far produces only surface scope readings when there are multiple quantifiers.

*Approach 1*: Following work in CCG, one might imagine that composition needn't match constituency; if the subject shifts and composes with the verb before the object we can get the other scoping.  (In CCG this is implemented using a function composition operation, not a type-shift.)

*Approach 2*: Someone interested in constituency might find this unsatisfying.  How could this be resolved using a type-shift?  One idea (due to Hendriks) is to build scope-taking shifts that operate on verb meanings.

In [ ]:
surface_shift_comb = %te L v_<e,<e,t>> : L f_<<e,t>,t> : L g_<<e,t>,t> : g(L y_e : f(L x_e : (v(x)(y))))
inverse_shift_comb = %te L v_<e,<e,t>> : L f_<<e,t>,t> : L g_<<e,t>,t> : f(L x_e : g(L y_e : (v(x)(y))))

inverse_shift_comb(saw.content).reduce_all()

In [ ]:
surface_shift_comb(saw.content).reduce_all()

Let's create a new composition system with both of these combinators used as typeshifts.

In [ ]:
system = lang.td_system.copy()
system.add_rule(lang.unary_factory(surface_shift_comb, "surface", typeshift=True))
system.add_rule(lang.unary_factory(inverse_shift_comb, "inverse", typeshift=True))
system.typeshift = True
lang.set_system(system)
system

In [ ]:
r = (someone * ((every * doctor) * saw))
display(*r.source_tree())
r

In [ ]:
for path in r:
    display(path.tree())

*Approach 3*: A final strategy would be to provide the first gq-shifter plus an even higher object type-lift that implements inverse scope.  This is effectively the combinator for Hendriks' inverse scope shifter with the order of arguments reversed.

In [ ]:
gq_lift_combinator = te("L f_<<e,t>,t> : L g_<e,<e,t>> : L x_e : f(L y_e : g(y)(x))")
gq_lift_combinator2 = te("L f_<<e,t>,t> : L g_<e,<e,t>> : L h_<<e,t>,t> : f(L y_e : h(L x_e : g(y)(x)))")

gq_lift_combinator2 #.type

In [ ]:
system = lang.td_system.copy()
system.add_rule(lang.unary_factory(gq_lift_combinator, "gq-lift-trans", typeshift=True))
system.add_rule(lang.unary_factory(gq_lift_combinator2, "gq-lift2-trans", typeshift=True))
system.typeshift = True
lang.set_system(system)
system

In [ ]:
r = (someone * ((every * doctor) * saw))
display(*r.source_tree())
r

In [ ]:
r[1].tree()